In [1]:
import pandas as pd

In [2]:
sales_history = pd.read_csv('../data/sales_history.csv')
merchandise_info = pd.read_csv('../data/merchandise_info.csv')
wholesale_prices = pd.read_csv('../data/processed_wholesale_prices.csv')

In [3]:
sales_history['date'] = pd.to_datetime(sales_history['date'])
recent_days = sales_history.date.unique()[-30:]
recent_discount = sales_history[(sales_history['discount']=='Y') & (sales_history['date'].isin(recent_days))].copy().reset_index(drop=True)
recent_discount.rename(columns={'sales_price':'discount_price'}, inplace=True)
recent_discount['normal_price']=0
recent_non_discount = sales_history[(sales_history['discount']=='N') & (sales_history['date'].isin(recent_days))]

In [4]:
for i in range(recent_discount.shape[0]):
    date = recent_discount.loc[i,'date']
    item_ID = recent_discount.loc[i,'item_ID']
    df = recent_non_discount[(recent_non_discount['item_ID']==item_ID) & (recent_non_discount['date']==date)]
    if df.shape[0]!=0:
        normal_price = df.iloc[0]['sales_price']
        recent_discount.loc[i,'normal_price'] = normal_price

print((recent_discount['normal_price']==0).sum()/recent_discount.shape[0])

0.025118805159538356


In [6]:
recent_discount = recent_discount[recent_discount['normal_price']!=0].copy().reset_index(drop=True)
recent_discount = recent_discount.merge(merchandise_info, on='item_ID', how='left')

In [8]:
category_discount = recent_discount.groupby(by='category_name').agg({'discount_price':'mean', 'normal_price':'mean'}).reset_index()
category_discount['avg_discount'] = category_discount['discount_price'] / category_discount['normal_price']
category_discount

,category_name,discount_price,normal_price,avg_discount
0,Aquatic Roots and Tubers,11.608621,18.468966,0.628547
1,Cauliflower,9.164706,13.247059,0.691829
2,Chili Peppers,6.254338,8.769863,0.713163
3,Edible Mushrooms,2.531492,3.645856,0.694348
4,Leafy Greens,3.624752,5.238944,0.691886
